# ___Coursera capstone notebook___

## ___W3 Assignment Part 1___

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import requests
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [ ]:
lst = pd.read_html(url)

In [220]:
df = pd.DataFrame(lst[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [221]:
df.drop(df[df['Borough'] == "Not assigned"].index, inplace = True)

In [222]:
df.reset_index(drop = True, inplace = True)

In [223]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list)
df = df.sample(frac=1).reset_index()
df['Neighbourhood']= df['Neighbourhood'].str.join(', ')

In [224]:
ind = df.loc[df['Neighbourhood'] == "Not assigned"].index[0]

In [225]:
df['Neighbourhood'].mask(df['Neighbourhood'] == "Not assigned", df.Borough[ind], inplace=True)

In [233]:
df.loc[ind]

Postcode                  M7A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 89, dtype: object

In [226]:
df.head(12)

,Postcode,Borough,Neighbourhood
0,M6E,York,Caledonia-Fairbanks
1,M6H,West Toronto,"Dovercourt Village, Dufferin"
2,M4P,Central Toronto,Davisville North
3,M6G,Downtown Toronto,Christie
4,M2M,North York,"Newtonbrook, Willowdale"
5,M5S,Downtown Toronto,"Harbord, University of Toronto"
6,M4L,East Toronto,"The Beaches West, India Bazaar"
7,M3A,North York,Parkwoods
8,M4G,East York,Leaside
9,M4B,East York,"Woodbine Gardens, Parkview Hill"


In [227]:
df.shape

(103, 3)

## ___W3 Assignment Part 2___

In [ ]:
df1 = pd.read_csv("http://cocl.us/Geospatial_data")
df1.rename(columns = {'Postal Code': "Postcode"}, inplace = True)

In [235]:
df_fin = pd.merge(df, df1, on='Postcode')

df_fin.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
1,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M6G,Downtown Toronto,Christie,43.669542,-79.422564
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
5,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049
6,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
7,M3A,North York,Parkwoods,43.753259,-79.329656
8,M4G,East York,Leaside,43.709060,-79.363452
9,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937


## ___W3 Assignment Part 3___

In [ ]:
tor_data = df_fin[df_fin.Borough.str.contains ('Toronto', case=False)].reset_index(drop=True)
tor_data.head()

In [ ]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor_data['Latitude'], tor_data['Longitude'], tor_data['Borough'], tor_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
CLIENT_ID = 'NZDQSGAU3EWXT0QIX0323PSYHEPD5TDVCLYX21O5BEBU22Q4' # your Foursquare ID
CLIENT_SECRET = 'QTOUCVJCIAYM5C2LX2KCO3UU4MZ2UPJAXMEEJDEXODD0BLPW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 #

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
tor_venues = getNearbyVenues(names=tor_data['Neighbourhood'],
                                   latitudes=tor_data['Latitude'],
                                   longitudes=tor_data['Longitude']
                                  )

In [ ]:
print(tor_venues.shape)
tor_venues.head()

In [ ]:
tor_venues.groupby('Neighbourhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

In [ ]:
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighbourhood'] = tor_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

#tor_onehot

In [ ]:
tor_grouped = tor_onehot.groupby('Neighbourhood').mean().reset_index()
tor_grouped.head()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [206]:
num_top_venues = 10

In [207]:
#num_top_venues = 10


indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = tor_grouped['Neighbourhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Hotel,Cosmetics Shop,Bakery,Restaurant,Asian Restaurant,American Restaurant
1,Berczy Park,Coffee Shop,Cheese Shop,Farmers Market,Steakhouse,Café,Beer Bar,Bakery,Cocktail Bar,Seafood Restaurant,Shopping Mall
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Restaurant,Italian Restaurant,Stadium,Burrito Place,Sandwich Place,Caribbean Restaurant,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Harbor / Marina,Sculpture Garden,Bar,Coffee Shop,Boat or Ferry,Boutique,Plane,Airport Terminal


In [202]:
kclusters = 5

In [208]:
# set number of clusters
#kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [209]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

tor_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Bookstore,Middle Eastern Restaurant,Sporting Goods Shop,Pizza Place,Bubble Tea Shop
1,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Candy Store,Italian Restaurant,Diner,Baby Store,Nightclub
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Italian Restaurant,Hotel,Gastropub,Gym,Café
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Gastropub,Breakfast Spot,Bakery,Clothing Store,Cosmetics Shop
4,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,0,Coffee Shop,Café,Steakhouse,Bar,Hotel,Cosmetics Shop,Bakery,Restaurant,Asian Restaurant,American Restaurant


In [ ]:
#tor_merged.dropna(inplace=True)

In [ ]:
#tor_merged.astype({'Cluster Labels': 'Int32'}).dtypes
#tor_merged['Cluster Labels'] = tor_merged['Cluster Labels'].astype(np.int32)

In [210]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighbourhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [211]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Bookstore,Middle Eastern Restaurant,Sporting Goods Shop,Pizza Place,Bubble Tea Shop
1,Downtown Toronto,0,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Candy Store,Italian Restaurant,Diner,Baby Store,Nightclub
2,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Italian Restaurant,Hotel,Gastropub,Gym,Café
3,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Gastropub,Breakfast Spot,Bakery,Clothing Store,Cosmetics Shop
4,Downtown Toronto,0,Coffee Shop,Café,Steakhouse,Bar,Hotel,Cosmetics Shop,Bakery,Restaurant,Asian Restaurant,American Restaurant
6,East Toronto,0,Health Food Store,Pub,Trail,Neighborhood,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
7,Downtown Toronto,0,Café,Vietnamese Restaurant,Bar,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Bakery,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dessert Shop
9,Downtown Toronto,0,Coffee Shop,Bakery,Park,Breakfast Spot,Pub,Café,Restaurant,Theater,Mexican Restaurant,Chocolate Shop
10,Central Toronto,0,Breakfast Spot,Park,Clothing Store,Food & Drink Shop,Hotel,Gym,Sandwich Place,Falafel Restaurant,Event Space,Ethiopian Restaurant
11,Central Toronto,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Bagel Shop,Mexican Restaurant,Diner,Dessert Shop,Park,Chinese Restaurant


In [212]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Gym,Yoga Studio,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [213]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Central Toronto,2,Garden,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [214]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Mexican Restaurant,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [215]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,4,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
21,Downtown Toronto,4,Park,Playground,Trail,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
